In [ ]:
!pip install Bio
!pip install py3Dmol
#!pip install altair_viewer
#!pip install altair_saver
#!pip install vl-convert-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.2 MB/s eta 0:00:00


In [ ]:
# Load libraries

import pandas as pd
import numpy as np
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import altair as alta
import py3Dmol

In [ ]:
# Read data

def read_fasta(file_path):
  records = {}
  for record in SeqIO.parse(file_path, "fasta"):
    records[record.id] = record
  return records

def read_genbank(file_path):
  records = {}
  with open(file_path, "r") as handle:
    for record in SeqIO.parse(handle, "genbank"):
      records[record.name] = record
  return records

reference_genome = read_fasta("Cal09_genome_reference.fa")
consensus_sequence = read_fasta("138B10-C8_masked_consensus.fa")
high_freq_variants = pd.read_csv("138B10-C8_high_freq_variants.txt", sep="\t", dtype={'CHROM': str})
low_freq_variants = pd.read_csv("138B10-C8_low_freq_variants.txt", sep="\t", dtype={'CHROM': str})
reference_annotation = read_genbank("Cal09_reference_annotation.txt")
consensus_annotation = read_genbank("138B10-C8_consensus_annotation.txt")

high_freq_variants['CHROM'] = high_freq_variants['CHROM'].fillna('NA')
low_freq_variants['CHROM'] = low_freq_variants['CHROM'].fillna('NA')

print("Reference Genome Segments:", reference_genome.keys())
print("Consensus Genome Segments:", consensus_sequence.keys())
print("Reference Annotation Segments:", reference_annotation.keys())
print("Consensus Annotation Segments:", consensus_annotation.keys())


Reference Genome Segments: dict_keys(['PB2', 'PB1', 'PA', 'HA', 'NP', 'NA', 'M', 'NS'])
Consensus Genome Segments: dict_keys(['PB2', 'PB1', 'PA', 'HA', 'NP', 'NA', 'M', 'NS'])
Reference Annotation Segments: dict_keys(['NC_026438', 'NC_026435', 'NC_026437', 'NC_026433', 'NC_026436', 'NC_026434', 'NC_026431', 'NC_026432'])
Consensus Annotation Segments: dict_keys(['PB2', 'PB1', 'PA', 'HA', 'NP', 'NA', 'M', 'NS'])


In [ ]:
# Identify mutations

def identify_mutations(reference, consensus):
  mutations = {}
  for segment in reference.keys():
    ref_seq = reference[segment].seq
    cons_seq = consensus[segment].seq
    segment_mutations = []
    for i, (ref_nt, cons_nt) in enumerate(zip(ref_seq, cons_seq)):
      if ref_nt != cons_nt:
        segment_mutations.append((i+1, ref_nt, cons_nt))
    mutations[segment] = segment_mutations
  return mutations

mutations = identify_mutations(reference_genome, consensus_sequence)

mutation_set = {f"{seg}_{pos}_{ref}_{alt}" for seg, muts in mutations.items() for pos, ref, alt in muts}

def filter_high_freq_mutations(mutations, high_freq_variants):
  high_freq_set = set()
  for index, row in high_freq_variants.iterrows():
    if row['HF CLASS'] == 'ALT':
      chrom = str(row['CHROM'])
      mutation_key = f"{row['CHROM']}_{row['POS']}_{row['REF']}_{row['ALT'].split(',')[0]}"
      if mutation_key not in mutations:
        high_freq_set.add((chrom, row['POS'], row['REF'], row['ALT'].split(',')[0]))
  return high_freq_set

filtered_high_freq_mutations = filter_high_freq_mutations(mutation_set, high_freq_variants)

def filter_low_freq_mutations(mutations, low_freq_variants):
  low_freq_set = set()
  for index, row in low_freq_variants.iterrows():
    if row['2HF CLASS'] == 'ALT':
      chrom = str(row['CHROM'])
      mutation_key = f"{row['CHROM']}_{row['POS']}_{row['REF']}_{row['ALT'].split(',')[0]}"
      if mutation_key not in mutations:
        low_freq_set.add((chrom, row['POS'], row['REF'], row['ALT'].split(',')[0]))
  return low_freq_set

filtered_low_freq_mutations = filter_low_freq_mutations(mutation_set, low_freq_variants)

print("Identified mutations between reference and consensus sequences:")
for segment, muts in mutations.items():
  print(f"{segment}: {muts}")

print("\nHigh-frequency mutations that did not pass filter:")
print(filtered_high_freq_mutations)

print("\nLow-frequency mutations detected in sample mixture:")
print(filtered_low_freq_mutations)

Identified mutations between reference and consensus sequences:
PB2: [(8, 'G', 'A'), (1445, 'A', 'G'), (1757, 'A', 'G'), (2172, 'G', 'A')]
PB1: [(708, 'A', 'G'), (1758, 'G', 'A')]
PA: [(1658, 'C', 'G')]
HA: [(598, 'T', 'C'), (622, 'C', 'A'), (640, 'G', 'A'), (715, 'G', 'A')]
NP: [(365, 'T', 'A'), (919, 'C', 'A')]
NA: []
M: []
NS: [(375, 'A', 'G')]

High-frequency mutations that did not pass filter:
set()

Low-frequency mutations detected in sample mixture:
{('PA', 1898, 'T', 'G'), ('PA', 2112, 'A', 'T'), ('M', 669, 'T', 'C'), ('PB1', 475, 'G', 'A'), ('PA', 2070, 'C', 'T'), ('NP', 90, 'A', 'G')}


In [ ]:
# Visualize mutations

mutations_list = []
for segment, muts in mutations.items():
  for pos, ref, alt in muts:
    mutations_list.append([segment, pos, ref, alt, 'Dominant'])

low_freq_list = []
for seg, pos, ref, alt in filtered_low_freq_mutations:
  low_freq_list.append([seg, pos, ref, alt, 'Low Frequency'])

mutations_df = pd.DataFrame(mutations_list + low_freq_list, columns=['Segment', 'Position', 'Ref', 'Alt', 'Frequency'])

segment_lengths = {segment: len(reference_genome[segment].seq) for segment in reference_genome.keys()}
segment_lines = pd.DataFrame([(segment, 1, length) for segment, length in segment_lengths.items()],
                             columns=['Segment', 'Start', 'End'])

segment_order = ['PB2', 'PB1', 'PA', 'HA', 'NP', 'NA', 'M', 'NS']

base = alta.Chart(mutations_df).encode(
    x='Position:Q',
    y=alta.Y('Segment:N', sort=segment_order),
    tooltip=['Segment', 'Position', 'Ref', 'Alt', 'Frequency']
)

dom_chart = base.mark_tick(color='#6BC38A', thickness=2, size=30, orient='vertical').transform_filter(
    alta.datum.Frequency == 'Dominant'
)

low_freq_chart = base.mark_tick(color='#6BC38A', thickness=2, size=30, orient='vertical', opacity=0.25).transform_filter(
    alta.datum.Frequency == 'Low Frequency'
)

segment_line_chart = alta.Chart(segment_lines).mark_rule(color='#436574').encode(
    x='Start:Q',
    x2='End:Q',
    y=alta.Y('Segment:N', sort=segment_order),
    size=alta.value(2)
)

final_chart = segment_line_chart + dom_chart + low_freq_chart

final_chart.properties(
    title='Nucleotide Mutations',
    width=1200,
    height=300
).configure_title(
    fontSize=16,
    fontWeight='normal',
    anchor='middle',
    color='#999999',
    font='Arial'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    labelFont='Arial',
    titleFont='Arial',
    titleFontWeight='normal',
    labelFontWeight='normal',
    titleColor='#999999',
    labelColor='#999999',
    grid=False,
    domain=False
).interactive()



alt.LayerChart(...)

In [ ]:
final_chart.save('138B10-C8_mutations.html')

In [ ]:
# Annotate protein changes

def extract_protein_sequences(annotation):
  protein_sequences = {}
  for record in annotation.values():
    for feature in record.features:
      if feature.type =='CDS':
        protein_id = feature.qualifiers.get('gene', [''])[0]
        protein_sequence = feature.qualifiers.get('translation', [''])[0]
        if protein_id:
          protein_sequences[protein_id] = protein_sequence
  return protein_sequences

reference_proteins = extract_protein_sequences(reference_annotation)
consensus_proteins = extract_protein_sequences(consensus_annotation)

def compare_protein_sequences(ref_proteins, cons_proteins):
  protein_changes = []
  for protein_id, ref_seq in ref_proteins.items():
    cons_seq = cons_proteins.get(protein_id, '')
    if not cons_seq:
      continue
    for i, (ref_aa, cons_aa) in enumerate(zip(ref_seq, cons_seq), start=1):
      if ref_aa != cons_aa:
        protein_changes.append([protein_id, i, ref_aa, cons_aa])
  return protein_changes

protein_changes = compare_protein_sequences(reference_proteins, consensus_proteins)
protein_changes_df = pd.DataFrame(protein_changes, columns=['Protein', 'Position', 'Ref_aa', 'Alt_aa'])

print(protein_changes_df)

  Protein  Position Ref_aa Alt_aa
0     PB2         3      R      K
1     PB2       482      K      R
2     PB2       586      K      R
3      PA       553      A      G
4      HA       200      S      P
5      HA       208      L      I
6      HA       214      A      T
7      NP       122      L      Q
8      NP       307      L      I


In [ ]:
# Visualize protein changes

protein_lengths = {protein: len(seq) for protein, seq in reference_proteins.items()}
segment_lines = pd.DataFrame([(protein, 1, length) for protein, length in protein_lengths.items()],
                             columns=['Protein', 'Start', 'End'])

protein_order = ['PB2', 'PB1', 'PA', 'PA-X', 'HA', 'NP', 'NA', 'M2', 'M1', 'NEP', 'NS1']

base_protein = alta.Chart(protein_changes_df).encode(
    x='Position:Q',
    y=alta.Y('Protein:N', sort=protein_order),
    tooltip=['Protein', 'Position', 'Ref_aa', 'Alt_aa']
)

protein_chart = base_protein.mark_tick(color='#436574', thickness=2, size=30, orient='vertical')

segment_line_chart = alta.Chart(segment_lines).mark_rule(color='#6BC38A').encode(
    x='Start:Q',
    x2='End:Q',
    y=alta.Y('Protein:N', sort=protein_order),
    size=alta.value(2)
)

final_protein_chart = segment_line_chart + protein_chart

final_protein_chart.properties(
    title='Amino Acid Changes',
    width=1200,
    height=300
).configure_title(
    fontSize=16,
    fontWeight='normal',
    anchor='middle',
    color='#999999',
    font='Arial'
).configure_axis(
    labelFontSize=12,
    titleFontSize=14,
    labelFont='Arial',
    titleFont='Arial',
    titleFontWeight='normal',
    labelFontWeight='normal',
    titleColor='#999999',
    labelColor='#999999',
    grid=False,
    domain=False
).interactive()

alt.LayerChart(...)

In [ ]:
final_protein_chart.save('138B10-C8_AA.html')

In [ ]:
# Visualize protein structure

#Cal09
pdb_ids = {
    'PB2': '3CW4',
    'PB1': '3AIG',
    'PA': '2ZNL',
    'HA': '3LZG',
    'NP': '2Q06',
    'NA': '3NSS',
    'M2': '2K1H',
    'M1': '1EA3',
    'NEP': '1PD3',
    'NS1': '3EU6'
}

#PR8
#pdb_ids = {
#    'PB2': '6QNW',
#    'PB1': '3AIG',
#    'PA': '6QPF',
#    'HA': '1RVX',
#    'NP': '2YMN',
#    'NA': '3CYE',
#    'M2': '2K1H',
#    'M1': '7JM3',
#    'NEP': '1PD3',
#    'NS1': '5NT2'
#}

def visualize_protein_structure(protein, pdb_id, changes):
  view = py3Dmol.view(query='pdb:' + pdb_id)
  view.setStyle({'cartoon': {'color': 'spectrum'}})

  for change in changes:
    pos, ref_aa, alt_aa = change
    label = f'{ref_aa}{pos}{alt_aa}'
    view.addStyle({'resi': str(pos)}, {'stick': {'colorscheme': 'redCarbon'}})
    view.addLabel(label, {'fontColor':'white', 'backgroundColor':'black'}, {'resi': str(pos)})

  view.zoomTo()
  view.show()

for protein, pdb_id in pdb_ids.items():
  changes = protein_changes_df[protein_changes_df['Protein'] == protein][['Position', 'Ref_aa', 'Alt_aa']].values
  visualize_protein_structure(protein, pdb_id, changes)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
# Visualize protein structure

pdb_ids = {
    'PB2': '3CW4',
    'PB1': '3AIG',
    'PA': '2ZNL',
    'HA': '3LZG',
    'NP': '2Q06',
    'NA': '3NSS',
    'M2': '2K1H',
    'M1': '1EA3',
    'NEP': '1PD3',
    'NS1': '3EU6'
}

def visualize_protein_structure(protein, pdb_id, changes):
  view = py3Dmol.view(query='pdb:' + pdb_id)
  view.setStyle({'cartoon': {'color': 'spectrum'}})

  for change in changes:
    pos, ref_aa, alt_aa = change
    label = f'{ref_aa}{pos}{alt_aa}'
    view.addStyle({'resi': str(pos)}, {'stick': {'colorscheme': 'redCarbon'}})
    view.addLabel(label, {'fontColor':'white', 'backgroundColor':'black'}, {'resi': str(pos)})

  view.zoomTo()
  view.render()
  html = view._make_html()
  with open(f'138B10-C8_{protein}_structure.html', 'w') as file:
    file.write(html)

for protein, pdb_id in pdb_ids.items():
  changes = protein_changes_df[protein_changes_df['Protein'] == protein][['Position', 'Ref_aa', 'Alt_aa']].values
  visualize_protein_structure(protein, pdb_id, changes)

from IPython.display import display, HTML

for protein in pdb_ids.keys():
  display(HTML(f'<a href="138B10-C8_{protein}_structure.html" target="_blank">138B10-C8_{protein}_structure.html</a>'))
